In [ ]:
##Importing from libraries
import spacy
spacy.load('en')
from spacy.en import English
parser = English()
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import random
import re
from gensim import corpora
import pickle
import gensim
import pyLDAvis.gensim

##Tokenizing and lower casing
def tokenize(text):
    A_tokens = []
    tokens = parser(text)
    
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('@'):
            continue
        else:
            A_tokens.append(token.lower_)
    return A_tokens


##Lematization
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

for w in ['cat', 'saved', 'happy']:
    print(w, get_lemma(w), get_lemma2(w))
    

##Downloading stopwords from nltk
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))


##Tokenization specifications
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

##Tweets check
text_data = []
input_File = open("dataset/combinedtweet.txt","r",encoding = "latin1").read()
tweet = input_File.split('\n')
document = []
for t in tweet:
      item = t.split('|')
      if len(item) > = 3:
          clean = re.sub(r"http\S+", "", item[2])
          document.append(clean)
print ("Cleaning is Done")
for line in document:
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)
print ("Done...")

len(text_data)

##Importing corpora for dictionary 
dictionary = corpora.Dictionary(text_data)

##dictionary processing
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

###########
##Clustering
num_clust = 4
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_clust = num_clust, id2word=dictionary, passes=25)
ldamodel.save('modelNew.gensim')

corpus = [dictionary.doc2bow(text) for text in text_data]
topics = ldamodel.print_topics(num_words=10)

for topic in topics:
    print(topic)
    
#Prediction class
def predict_class(new_tweet):
    new_tweet = prepare_text_for_lda(new_tweet)
    new_tweet_bow = dictionary.doc2bow(new_tweet)
    pred =ldamodel.get_document_topics(new_tweet_bow)

    max=pred[0][1]
    label=0
    for i in range(1,4):
        if (pred[i][1]>max):
            max= pred[i][1]
            label=i
    return label

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('modelNew.gensim')


lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.show(lda_display)

lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

##Writing data
csv_file_file = open("dataset/health_tweets_labeled.csv_file", "a") 
columnTitleRow = "tweet, class\n"
csv_file.write(columnTitleRow)

for i in range(13618,len(text_data)):
    tweet = text_data[i]
    tweet_str = ' '.join(str(e) for e in tweet)
    row = tweet_str.replace(",","") + "," + str(predict_class(tweet_str)) + "\n"
    csv_file.write(row)
csv_file.close()